<a href="https://colab.research.google.com/github/umersolangi/Nodebook/blob/main/Ml_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langgraph "langchain[anthropic]"


In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install -qU langchain-openai

In [ ]:
!pip install -qU pypdf

In [ ]:
!pip install langgraph langchain  faiss-cpu

In [ ]:
!pip install openai

LOADER:


In [ ]:
from langchain_community.document_loaders import PyPDFLoader


In [ ]:
pdfref = PyPDFLoader("/content/my_questions.pdf")
docs = pdfref.load()

In [ ]:
from openai import OpenAI
import os
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

CHunking

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
TextSplitter= RecursiveCharacterTextSplitter(chunk_size=1100,chunk_overlap=250)
chunks = TextSplitter.split_documents(docs)

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large")
store = InMemoryVectorStore(embeddings)
store.add_documents(chunks)

In [ ]:
from langgraph.graph import StateGraph, END

In [ ]:
from langchain_openai import OpenAI
def generate_with_source(query, context):
    prompt = f"""
    Answer the following question based on the provided context. If the answer comes from the document, mention that it was retrieved from the PDF. If the answer is generated by you (the LLM), mention that it comes from your internal knowledge base.

    Question: {query}

    Context: {context}


    Answer:
    """
    llm = OpenAI(model="gpt-3.5-turbo-instruct")

    response = llm.invoke(prompt)

    return response

In [ ]:
def Answerable(query:int):
    results =store.similarity_search(query, k=3)
    context = "\n".join([f"{d.page_content} (Source: {d.metadata.get('source')})" for d in results])
    answer = generate_with_source(query, context)
    return answer


In [ ]:
query = "What is Ohm’s Law?"
answer = Answerable(query)
print(answer)

In [ ]:
query = "Define Machine Learning"
answer = Answerable(query)
print(answer)

In [ ]:
from typing import TypedDict, Annotated, List
from langchain_core.documents import Document
import operator

class GraphState(TypedDict):
    question: str
    context: Annotated[List[Document], operator.add]
    answer: str

In [ ]:
from langchain_openai import OpenAI
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
def retrieve(state: GraphState):
    question = state["question"]
    results = store.similarity_search(question, k=3)
    state["context"] = results
    return state

def generate(state: GraphState):
    question = state["question"]
    context = "\n".join([f"{d.page_content} (Source: {d.metadata.get('source')})" for d in state["context"]])
    answer = generate_with_source(question, context)
    state["answer"] = answer
    return state

In [ ]:


workflow = StateGraph(GraphState)

workflow.add_node("retrieve", retrieve)
workflow.add_node("generate", generate)
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.set_finish_point("generate")
app = workflow.compile()

In [ ]:
app.invoke()
